In [ ]:
import numpy as np
import inspect # to prvent using private methods in class

# Implementing NN output with one hidden layer

In [ ]:
def sigmoid(Z):
  return (1/(1 + np.exp(-Z)))

In [ ]:
def Compute_Z(X, W, B):
  z = np.dot(X, W) + B
  return z

In [ ]:
def OneHiddenLayerNN(X,W1,B1,W2,B2):
  Z1 = Compute_Z(X,W1,B1)
  A1 = sigmoid(Z1)
  Z2 = Compute_Z(A1,W2,B2)
  A2 = sigmoid(Z2)
  return A2

implement gradient descent for neural network with one hidden layer

and for now assume that we're doing binary classification so the cost will be
(y_hat - y)**2

note that:


*   y_hat is A2
*   keepdims = true in mumpy make sure that the result will not be (n,) it will be (n, m)

*   g' refers to the drevative of the activation function





In [ ]:
def NNGDFor1HiddenUnit(X, Y, W1, B1, W2, B2, alpha, iter_number):
  num_samples, num_features = X.shape
  for iter in range(iter_number):
    A1 = sigmoid(Compute_Z(X,W1,B1))
    A2 = OneHiddenLayerNN(X,W1,B1,W2,B2)

    DZ2 = A2 - Y
    DW2 = (1/num_samples) * np.dot(A1.T,DZ2)
    DB2 = (1/num_samples) * np.sum(DZ2,axis=1,keepdims=True)
    DZ1 = np.dot(DZ2,W2.T) * (A1 * (1 - A1))
    DW1 = (1/num_samples) * np.dot(X.T,DZ1)
    DB1 = (1/num_samples) * np.sum(DZ1,axis=1,keepdims=True)

    W1 = W1 - alpha * DW1
    W2 = W2 - alpha * DW2
    B1 = B1 - alpha * DB1
    B2 = B2 - alpha * DB2

  return W1, W2, B1, B2

Example:

In [ ]:
# Example usage:
input_size = 2
hidden_size = 4
output_size = 1
# Initialize weights and biases
W1 = np.random.uniform(size=(input_size, hidden_size))
B1 = np.random.uniform(size=(1, hidden_size))
W2 = np.random.uniform(size=(hidden_size, output_size))
B2 = np.random.uniform(size=(1, output_size))

X = np.array([[0.1, 0.2], [0.3, 0.4]])
Y = np.array([[0.5], [0.7]])
W1_new, W2_new, b1_new, b2_new = NNGDFor1HiddenUnit(X, Y, W1, B1, W2, B2, 0.1, 1000)

output = OneHiddenLayerNN(X,W1_new,b1_new,W2_new,b2_new)
print(output)

[[0.5000008 ]
 [0.69999925]]


# better implementation

In [ ]:
class OneHiddenLayerNN:
    def __init__(self,input_size, hidden_size, output_size, alpha):
        self.W1 = np.random.uniform(size=(input_size, hidden_size))
        self.B1 = np.random.uniform(size=(1, hidden_size))
        self.W2 = np.random.uniform(size=(hidden_size, output_size))
        self.B2 = np.random.uniform(size=(1, output_size))
        self.alpha = alpha

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def compute_Z(self, X, W, B):
        return np.dot(X, W) + B

    def forward_propagation(self, X):
        Z1 = self.compute_Z(X, self.W1, self.B1)
        A1 = self.sigmoid(Z1)
        Z2 = self.compute_Z(A1, self.W2, self.B2)
        A2 = self.sigmoid(Z2)
        return A1, A2

    def train(self, X, Y, iter_number):
        num_samples, num_features = X.shape

        for iter in range(iter_number):
            A1, A2 = self.forward_propagation(X)
            print(A2.shape)
            print(Y.shape)
            DZ2 = A2 - Y
            print("DZ2 shape is:",DZ2.shape)
            DW2 = (1 / num_samples) * np.dot(A1.T, DZ2)
            DB2 = (1 / num_samples) * np.sum(DZ2, axis=0, keepdims=True)
            DZ1 = np.dot(DZ2, self.W2.T) * (A1 * (1 - A1))
            DW1 = (1 / num_samples) * np.dot(X.T, DZ1)
            DB1 = (1 / num_samples) * np.sum(DZ1, axis=0, keepdims=True)

            self.W1 -= self.alpha * DW1
            self.W2 -= self.alpha * DW2
            self.B1 -= self.alpha * DB1
            self.B2 -= self.alpha * DB2

        return self.W1, self.W2, self.B1, self.B2


In [ ]:
# # Example usage:
input_size = 2
hidden_size = 4
output_size = 1

X2 = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.7]])
Y2 = np.array([[0.5], [0.7], [0.9]])
nn2 = OneHiddenLayerNN(input_size, hidden_size, output_size, 2)
W1_new2, W2_new2, b1_new2, b2_new2 = nn2.train(X2, Y2, 1)
A1, A2 = nn2.forward_propagation(X2)
print("predicted:" , A2)
print("actual:" , Y2)

(3, 1)
(3, 1)
DZ2 shape is: (3, 1)
predicted: [[0.75961034]
 [0.77386876]
 [0.78940534]]
actual: [[0.5]
 [0.7]
 [0.9]]


to prevent using private methods

In [ ]:
def private_method(method):
    def wrapper(self, *args, **kwargs):
        stack = inspect.stack()
        # Check if the caller is a method of the same class
        if stack[1].function not in dir(self):
            raise AttributeError(f"{method.__name__} is a private method.")
        return method(self, *args, **kwargs)
    return wrapper
# this will make that move: "obj._MyClass__private_method()"  # raise an AttributeError

# Deep L Neural Networks

In [ ]:
class DeepNNOneOutput:
    def __init__(self, DNN_Layers, alpha):
        self.W = []
        self.B = []
        self.costHistory = []
        self.DNN_Layers = DNN_Layers
        self.alpha = alpha

    # helper functions
    def __initWeights(self,X):
        X_size = X.shape[1]
        for i in range(len(self.DNN_Layers)):
            if i == 0:
                self.W.append(np.random.uniform(size=(X_size, self.DNN_Layers[i])))
                self.B.append(np.random.uniform(size=(1, self.DNN_Layers[i])))
            else:
                self.W.append(np.random.uniform(size=(self.DNN_Layers[i-1],self.DNN_Layers[i])))
                self.B.append(np.random.uniform(size=(1, self.DNN_Layers[i])))
            # print(self.W[i].shape)

    def __sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def __compute_Z(self, X, W, B):
        return np.dot(X, W) + B

    def __compute_cost(self, Y, Y_hat):
        m = Y.shape[1]
        cost = -(1 / m) * np.sum(Y * np.log(Y_hat) + (1 - Y) * np.log(1 - Y_hat))
        return np.squeeze(cost) # np.squeeze used to transform the result from (1, 1) ig.[[5]] to () ig. 5

    def forward_propagation(self, X):
        # intialize the weights
        if not self.W:
            self.__initWeights(X)
        # to store every layer output
        A = [X] # Initialize A with the input X
        for i in range(len(self.DNN_Layers)):
            Z = self.__compute_Z(A[i], self.W[i], self.B[i])
            A.append(self.__sigmoid(Z))
        return A[1:] # Return the outputs of all layers except the input

    def train(self, X, Y, iter_number):
        num_samples, num_features = X.shape

        for iter in range(iter_number):
          A = self.forward_propagation(X)
          if iter % 100 == 0:
              cost = self.__compute_cost(Y, A[-1])
              self.costHistory.append(cost)

          # Backpropagation
          DA = A[-1] - Y
          # print("A[-1] shape is: " ,A[-1].shape)
          # print("Y shape is: " ,Y.shape)
          # print("DA shape is: ",DA.shape)
          for i in reversed(range(len(self.DNN_Layers))):
            # print("A[i] shape is:",A[i].shape)
            DZ = DA * (A[i] * (1 - A[i]))
            DW = (1 / num_samples) * np.dot(A[i-1].T, DZ )
            DB = (1 / num_samples) * np.sum(DZ, axis=0, keepdims=True)
            if i != 0:
                DA = np.dot(DZ, self.W[i].T)
            self.W[i] -= self.alpha * DW
            self.B[i] -= self.alpha * DB

    def predict(self, X):
        A = self.forward_propagation(X)
        Y_hat = A[-1]
        predictions = (Y_hat > 0.5).astype(int)  # Convert probabilities to binary output
        return predictions

    def accuracy(self, X, Y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == Y)
        return accuracy

In [ ]:
# # Example usage:
# DNN_Layers = [5, 5, 3, 1] #overfit
DNN_Layers = [5,1]
X2 = np.array([[0.1, 0.2, 0.2], [0.3, 0.4, 0.3], [0.5, 0.7, 0.6],[0.8, 0.9, 0.7]])
Y2 = np.array([[0.5], [0.7], [0.8], [0.9]])
print(X2.shape)
print(Y2.shape)
hnn = DeepNNOneOutput(DNN_Layers, 10)
hnn.train(X2, Y2, 1000)
A2 = hnn.predict(X2)
print(A2)
# print(hnn.costHistory)

(4, 3)
(4, 1)
[[0.50936711]
 [0.67876348]
 [0.82628345]
 [0.88636923]]


testing on a imdb dataset

In [ ]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
print(x_train.shape)
print(y_train.shape)
max_length = 500
x_train_padded = pad_sequences(x_train, maxlen=max_length)
x_test_padded = pad_sequences(x_test, maxlen=max_length)

# Reshape the labels
y_train_reshaped = np.array(y_train).reshape(-1, 1)
y_test_reshaped = np.array(y_test).reshape(-1, 1)

# Check the shapes
print(x_train_padded.shape)  # Expected: (25000, 500)
print(y_train_reshaped.shape)  # Expected: (25000, 1)

(25000,)
(25000,)
(25000, 500)
(25000, 1)


In [ ]:
IMDB_DNN_Layers = [50,1]
hnn_imdb = DeepNNOneOutput(IMDB_DNN_Layers, 10)
hnn_imdb.train(x_train_padded, y_train_reshaped, 1000)
print(hnn_imdb.costHistory)

[320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695, 320452.6722720695]


In [ ]:
IMDB_ACC = hnn_imdb.accuracy(x_test_padded, y_test_reshaped)
print(IMDB_ACC)

0.5


In [ ]:
IMDB_DNN_Layers = [50, 25, 1]
hnn_imdb_model2 = DeepNNOneOutput(IMDB_DNN_Layers, 10)
hnn_imdb_model2.train(x_train_padded, y_train_reshaped, 1000)
IMDB_ACC_2 = hnn_imdb_model2.accuracy(x_test_padded, y_test_reshaped)
print(IMDB_ACC_2)

0.5


In [ ]:
IMDB_DNN_Layers = [50, 25, 1]
hnn_imdb_model3 = DeepNNOneOutput(IMDB_DNN_Layers, 0.1)
hnn_imdb_model3.train(x_train_padded, y_train_reshaped, 1000)
IMDB_ACC_3 = hnn_imdb_model3.accuracy(x_test_padded, y_test_reshaped)
print(IMDB_ACC_3)

0.5


normalizing the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xTrain_normalized = scaler.fit_transform(x_train_padded)
xTest_normalized = scaler.fit_transform(x_test_padded)

In [ ]:
IMDB_DNN_Layers = [100, 50, 25, 1]
hnn_imdb_model4 = DeepNNOneOutput(IMDB_DNN_Layers, 0.01)
hnn_imdb_model4.train(xTrain_normalized, y_train_reshaped, 1000)
IMDB_ACC_4 = hnn_imdb_model4.accuracy(xTest_normalized, y_test_reshaped)
print(IMDB_ACC_4)

0.5


# Changing in the structure of DeepNN Class

In [ ]:
class DeepNNONEOUT:
    def __init__(self, DNN_Layers, alpha, threshold):
        self.W = []
        self.B = []
        self.costHistory = []
        self.threshold = threshold
        self.DNN_Layers = DNN_Layers
        self.alpha = alpha

    # Helper functions
    def __initWeights(self, X):
        np.random.seed(1)
        X_size = X.shape[1]
        for i in range(len(self.DNN_Layers)):
            if i == 0:
                self.W.append(np.random.randn(X_size, self.DNN_Layers[i]) )
                self.B.append(np.zeros((1, self.DNN_Layers[i])))
            else:
                self.W.append(np.random.randn(self.DNN_Layers[i-1], self.DNN_Layers[i]))
                self.B.append(np.zeros((1, self.DNN_Layers[i])))

    def __sigmoid(self, Z):
        Z = np.clip(Z, -500, 500)  # Clipping values for numerical stability
        return 1 / (1 + np.exp(-Z))

    def __compute_Z(self, X, W, B):
        return np.dot(X, W) + B

    def __compute_cost(self, Y, Y_hat):
        m = Y.shape[0]
        cost = -(1 / m) * np.sum(Y * np.log(Y_hat) + (1 - Y) * np.log(1 - Y_hat))
        return np.squeeze(cost)

    def forward_propagation(self, X):
        A = [X]
        for i in range(len(self.DNN_Layers)):
            Z = self.__compute_Z(A[i], self.W[i], self.B[i])
            A.append(self.__sigmoid(Z))
        return A

    def train(self, X, Y, iter_number):
        if not self.W:
            self.__initWeights(X)

        num_samples = X.shape[0]

        for iter in range(iter_number):
            A = self.forward_propagation(X)
            if iter % 100 == 0:
                cost = self.__compute_cost(Y, A[-1])
                self.costHistory.append(cost)

            # Backpropagation
            DA = A[-1] - Y
            for i in reversed(range(len(self.DNN_Layers))):
                DZ = DA * (A[i+1] * (1 - A[i+1]))
                DW = (1 / num_samples) * np.dot(A[i].T, DZ)
                DB = (1 / num_samples) * np.sum(DZ, axis=0, keepdims=True)
                if i != 0:
                    DA = np.dot(DZ, self.W[i].T)

                self.W[i] -= self.alpha * DW
                self.B[i] -= self.alpha * DB

    def predict(self, X):
        A = self.forward_propagation(X)
        Y_hat = A[-1]
        predictions = (Y_hat > self.threshold).astype(int)
        return predictions, Y_hat

    def accuracy(self, X, Y):
        predictions, _ = self.predict(X)
        accuracy = np.mean(predictions == Y)
        return accuracy

    def get_W_B(self):
        return self.W, self.B


In [ ]:
# Initialize and train the model
model = DeepNNONEOUT(DNN_Layers=[10, 5, 1], alpha=0.01, threshold=0.43)  # Example architecture
model.train(x_train_padded, y_train_reshaped, iter_number=1000)

# Compute accuracy on training set
train_accuracy = model.accuracy(x_train_padded, y_train_reshaped)
print(f"Training accuracy: {train_accuracy * 100:.2f}%")

# Compute accuracy on test set
test_accuracy = model.accuracy(x_test_padded, y_test_reshaped)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

predictions, y_hat = model.predict(x_test_padded)
print(predictions)
print("Actual")
print(y_test_reshaped)

print(y_hat)

Training accuracy: 50.54%
Test accuracy: 50.03%
[[1]
 [1]
 [0]
 ...
 [1]
 [0]
 [1]]
Actual
[[0]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]
[[0.46275377]
 [0.47110205]
 [0.32701022]
 ...
 [0.44247872]
 [0.42845375]
 [0.48383076]]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage import io, color, transform, feature

In [ ]:
# Define the model
model0 = models.Sequential([
    layers.Dense(64, activation='sigmoid'),
    layers.Dense(10, activation='sigmoid')
])
# Compile the model
model0.compile(optimizer=tf.keras.optimizers.SGD(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model0.fit(x_train_padded, y_train_reshaped, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 2s 3ms/step - loss: 0.8134 - accuracy: 0.5001 - val_loss: 0.7235 - val_accuracy: 0.4832
Epoch 2/10
625/625 [==============================] - 2s 2ms/step - loss: 0.7127 - accuracy: 0.5038 - val_loss: 0.7072 - val_accuracy: 0.5112
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.7029 - accuracy: 0.5049 - val_loss: 0.7010 - val_accuracy: 0.5026
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.6991 - accuracy: 0.5120 - val_loss: 0.6980 - val_accuracy: 0.4988
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6970 - accuracy: 0.5142 - val_loss: 0.6965 - val_accuracy: 0.5044
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.6955 - accuracy: 0.5198 - val_loss: 0.6956 - val_accuracy: 0.4982
Epoch 7/10
625/625 [==============================] - 2s 2ms/step - loss: 0.6955 - accuracy: 0.5113 - val_loss: 0.6951 - val_accuracy: 0.5162
Epoch 

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model0.evaluate(xTest_normalized, y_test)
# Format the test accuracy to display as 100 percent
formatted_test_accuracy = "{:.3%}".format(test_accuracy)

# Print the formatted test accuracy
print("Test Accuracy:", formatted_test_accuracy)

782/782 [==============================] - 2s 2ms/step - loss: 0.7037 - accuracy: 0.5051
Test Accuracy: 50.508%


it seems that the data needs more preprossing on it but the most important thing to me that my model works fine